## GPU Programming: Perform vector and matrix operations in CUDA

### Environment Setup

You can run this notebook on either Colab or clone the github repo to your virtual machine with GPU.

In [ ]:
### This script is used to clone the repository to the google drive and setup the directory
from google.colab import drive
import os

drive_path = '/content/drive' 
drive.mount(drive_path)
workspace_path = os.path.join(drive_path, "MyDrive/workspace/11868hw")
!mkdir -p {workspace_path}
%cd {workspace_path}
git_repo_name = "llmsys_code_examples"
repo_path = os.path.join(workspace_path, git_repo_name)
if os.path.isdir(os.path.join(repo_path, ".git")):
  %cd {git_repo_name}
  !git pull
else: 
  !git clone https://github.com/llmsystem/llmsys_code_examples.git
%cd {repo_path}/simple_cuda_demo

### Section1: Compile and Run CUDA Code

We will implment a program to add two vectors in cuda. 
The following code snippt is from `example_vector_add.cu`. 

```cpp

__global__ void VecAddKernel(int* A, int* B, int* C, int n) {
  // blockDim is size of block along x-axis
  // blockIdx is the index of the current thread's block
  // threadIdx is the index of the current thread within the block
  int i = blockDim.x * blockIdx.x + threadIdx.x;
  if (i < n) {
    C[i] = A[i] + B[i];
  }
}

```

Please check the full code in `example_vector_add.cu`. 

Run the following command: 

In [ ]:
# Remove ! if running on the terminal of your vm
# Compile the codes for matrix addition
!nvcc -o vecadd example_vector_add.cu

Run the following command to check the result: 

In [ ]:
!./vecadd

In `example_matadd.cu` and `example_matmul2.cu`, there are example codes for matrix addition and matrix multiplication. 

The following code snippets are from `example_matadd.cu` to perform matrix multiplication:

```cpp

__global__ void matrixAdd(const int * a, const int * b,
                          int * c, int N) {
  // Compute each thread's global row and column index
  int row = blockIdx.x * blockDim.x + threadIdx.x;
  int col = blockIdx.y * blockDim.y + threadIdx.y;
  
  // Iterate over row, and down column
  if (row < N && col < N) {
    c[row * N + col] = a[row * N + col] + b[row * N + col];
  }
}

```

Run the following command to check the results.

In [ ]:
# Remove ! if running on the terminal of your vm
# Compile the codes for matrix addition
!nvcc -o matadd example_matadd.cu

In [ ]:
# Run the codes
!./matadd

The following code snippets illustrate the multiplication of two matrices. 

```c++

__global__ void MatmulKernel(const float* a, const float* b, float* out, 
                             int M, int N, int P) {
  // Compute each thread's global row and column index
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  if (idx >= M * P) return;
  int row = idx / P;
  int col = idx % P;
  if (row < M && col < P) {
    // Calculate the matrix multiplication for row in matrix a and col in matrix b
    float sum = 0.0;
    for (int i = 0; i < N; i++) {
      sum += a[row * N + i] * b[i * P + col];
    }
    out[row * P + col] = sum;
  }
}

```

```c++
__global__ void MatmulKernel(const int *a, const int *b, int *c, int M, int N, int P) {
  // Compute each thread's global row and column index
  int row = blockIdx.x * blockDim.x + threadIdx.x;
  int col = blockIdx.y * blockDim.y + threadIdx.y;
  if (row >= M || col >= P) return;
  // Iterate over row, and down column
  c[row * P + col] = 0;
  for (int k = 0; k < N; k++) {
    // Accumulate results for a single element
    c[row * P + col] += a[row * N + k] * b[k * P + col];
  }
}
```

Run the following command. 

In [ ]:
# Compile the codes for matrix addition
!nvcc -o matmul example_matmul2.cu

In [ ]:
# Run the codes
!./matmul

### Section2: Run CUDA Codes with Python Calls

This section shows three demos, including the vector addition, window summation and matrix multiplication implemented in CUDA. We call these CUDA functions within the python codes, which follows the same recipe in our assignment1.

In [ ]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.3 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1-cp310-cp310-linux_x86_64.whl size=661205 sha256=87b18480433e8cc46e8387b1463bb1b1fa14bd3c2f1dcf7f9a33451b88e59872
  Stored in directory: /root/.cache/pip/wheels/12/34/d2/9a349255a4eca3a486d82c79d21e138ce2ccd90f414d9d72b8
Successfully built pycuda


## CUDA Example for Vector Add

We demonstrate how we can call C function in python file with ctypes with `VecAddCPU` function.

We also demonstrate two ways to write CUDA codes which can be called in python functions. The difference between them is that we create CUDA memory and copy the data to CUDA device by `pycuda` package in the `VecAddCUDA` function and `cudaMemcpy` in cpp codes in `VecAddCUDA2` function.

We use `pycuda` in assignment1 to call CUDA kernel functions.

In [ ]:
!nvcc -o vector_add.so --shared example_vector_add.cu -Xcompiler -fPIC

```python
# Define the argument types and return types
lib.VecAddCUDA.argtypes = [
    ctypes.POINTER(ctypes.c_int),
    ctypes.POINTER(ctypes.c_int),
    ctypes.POINTER(ctypes.c_int),
    ctypes.c_int,
]

lib.VecAddCUDA.restype = None

# Load the arrays to CUDA device
a_gpu = gpuarray.to_gpu(a)
b_gpu = gpuarray.to_gpu(b)
c_gpu = gpuarray.to_gpu(cgpu)

# Call the C wrapper function with CUDA kernel
lib.VecAddCUDA(
    ctypes.cast(a_gpu.ptr, ctypes.POINTER(ctypes.c_int)),
    ctypes.cast(b_gpu.ptr, ctypes.POINTER(ctypes.c_int)),
    ctypes.cast(c_gpu.ptr, ctypes.POINTER(ctypes.c_int)),
    ctypes.c_int(size)
)

# Load the gpuarray back to array in the host device
cgpu = c_gpu.get()
print(f"After offload: {cgpu}, {type(cgpu)}")
```

In [ ]:
!python test_vector_add.py

Input a: [7 5 1 7 6 4 7 3 9 3]
Input b: [8 4 2 6 5 1 7 7 8 6]
Numpy add: [15  9  3 13 11  5 14 10 17  9], <class 'numpy.ndarray'>
CPU add: [15  9  3 13 11  5 14 10 17  9], <class 'numpy.ndarray'>
GPU add: [15  9  3 13 11  5 14 10 17  9], <class 'pycuda.gpuarray.GPUArray'>
After offload: [15  9  3 13 11  5 14 10 17  9], <class 'numpy.ndarray'>
GPU add2: [15  9  3 13 11  5 14 10 17  9], <class 'numpy.ndarray'>


## CUDA Example for Window Sum

Demo of Window Sum to get to know about synchronization in CUDA.

In [ ]:
!nvcc -o window_sum.so --shared example_window_sum.cu -Xcompiler -fPIC

In [ ]:
!python test_window_sum.py

Input: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
Numpy window sum: [15. 20. 25. 30. 35. 40. 45. 50.]
GPU simple window sum: [15. 20. 25. 30. 35. 40. 45. 50.]
GPU shared window sum: [15. 20. 25. 30. 35. 40. 45. 50.]


## CUDA Example for Matrix Multiplication

Demo of matrix multiplication.

In [ ]:
!nvcc -o matmul.so --shared example_matmul.cu -Xcompiler -fPIC

In [ ]:
!python test_matmul.py

Input a: [[1. 2. 2. 1.]
 [2. 2. 2. 2.]
 [2. 2. 2. 1.]
 [1. 2. 1. 1.]]
Input b: [[1. 1.]
 [1. 2.]
 [1. 2.]
 [2. 1.]]
Numpy matmul: [[ 7. 10.]
 [10. 12.]
 [ 8. 11.]
 [ 6.  8.]], <class 'numpy.ndarray'>
GPU matmul: [[ 7. 10.]
 [10. 12.]
 [ 8. 11.]
 [ 6.  8.]], <class 'pycuda.gpuarray.GPUArray'>
After offload: [[ 7. 10.]
 [10. 12.]
 [ 8. 11.]
 [ 6.  8.]], <class 'numpy.ndarray'>
Compare result: 0.0
